# Various Classification Models

In [43]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.neural_network import MLPClassifier
import seaborn as sns
import matplotlib.pyplot as plt
import os
import glob

In [45]:
path = os.getcwd()
csv_files = glob.glob(os.path.join(path, "./data/Combined_EMG_data*"))
data_list = []
X_list = []
y_list = []

for i in csv_files:
      
    df = pd.read_csv(i)
    data_list.append(df)

for j in data_list:
    X = j[['EMG Sensor 1', 'EMG Sensor 2', 'EMG Sensor 3', 'EMG Sensor 4']].to_numpy().reshape(-1,4)
    y = j['Mode'].to_numpy()
    X_list.append(X)
    y_list.append(y)
    
X = np.vstack(X_list)
y = np.hstack(y_list)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

## KNN

In [47]:
m = []

for i in (x+1 for x in range(0, 25)):
    KNN = KNeighborsClassifier(n_neighbors=i)
    KNN.fit(X_train,y_train)
    scores = cross_val_score(KNN, X_train, y_train, cv=5)
    print(f"K: {i} , Mean: {scores.mean()} , Std: {scores.std()}")
    m.append(scores.mean())

print(f"Max score: {max(m)}")

K: 1 , Mean: 0.8605995525727069 , Std: 0.016013487201187237
K: 2 , Mean: 0.8673020134228187 , Std: 0.012902561942496912
K: 3 , Mean: 0.8820402684563758 , Std: 0.018285485851848762
K: 4 , Mean: 0.8793646532438479 , Std: 0.017439207559177668
K: 5 , Mean: 0.8820581655480983 , Std: 0.024854737151108693
K: 6 , Mean: 0.8766979865771812 , Std: 0.019564116404566068
K: 7 , Mean: 0.8793914988814319 , Std: 0.022227611516120998
K: 8 , Mean: 0.8740313199105145 , Std: 0.020252629246267615
K: 9 , Mean: 0.880724832214765 , Std: 0.024735833272451876
K: 10 , Mean: 0.8820939597315436 , Std: 0.03078079176321168
K: 11 , Mean: 0.8860939597315436 , Std: 0.03327338903822806
K: 12 , Mean: 0.8887874720357942 , Std: 0.029657020072687838
K: 13 , Mean: 0.8901118568232663 , Std: 0.0281965825968855
K: 14 , Mean: 0.891463087248322 , Std: 0.031972196380308605
K: 15 , Mean: 0.8860939597315436 , Std: 0.03662433755997683
K: 16 , Mean: 0.8860939597315436 , Std: 0.03132088716917636
K: 17 , Mean: 0.8847427293064877 , Std: 0

In [48]:
KNN = KNeighborsClassifier(n_neighbors=15)

KNN.fit(X_train, y_train)
predictions = KNN.predict(X_test)
probs = KNN.predict_proba(X_test)
score = KNN.score(X_test, y_test)
# print(f"X_test: \n{X_test}")
# # print(f"Predictions: \n{predictions}")
# print(f"Probabilities: \n{probs}")
print(f"K: 15")
print(f"Score: {score}")

K: 15
Score: 0.9144385026737968


In [55]:
none = np.array([0.26,0.07,0.1,0.32]).reshape(-1,4)
pred = KNN.predict(none)
print(pred)

index = np.array([0.9, 0.1, 0.4, 0.5]).reshape(-1,4)
pred = KNN.predict(index)
print(pred)

middle = np.array([1.6, 0.2, 1.5, 1.6]).reshape(-1,4)
pred = KNN.predict(middle)
print(pred)

fourth = np.array([1.9, 0.3, 0.6, 1.1]).reshape(-1,4)
pred = KNN.predict(fourth)
print(pred)

pinky = np.array([0.6, 0.2, 0.6, 1.1]).reshape(-1,4)
pred = KNN.predict(pinky)
print(pred)

[0]
[1]
[2]
[3]
[4]


## Random Forest

In [37]:
RFC = RandomForestClassifier()

values = range(1,51)

parameter_grid = {'n_estimators': values, 'max_depth': values}

grid = GridSearchCV(estimator=RFC, param_grid=parameter_grid)

grid.fit(X_train, y_train)


GridSearchCV(estimator=RandomForestClassifier(),
             param_grid={'max_depth': range(1, 51),
                         'n_estimators': range(1, 51)})

In [38]:
print(grid.best_params_)

{'max_depth': 4, 'n_estimators': 35}


In [52]:
RFC = RandomForestClassifier(n_estimators=35, max_depth=4)

RFC.fit(X_train, y_train)

y_pred = RFC.predict(X_test)

print(f"Features importances: \n{RFC.feature_importances_}")
print(f"Confusion matrix: \n{confusion_matrix(y_test, y_pred)}") # predicted vertical, actual horizontal
print(f"Score: {RFC.score(X_test, y_test)}")

Features importances: 
[0.37098283 0.05589384 0.24046042 0.3326629 ]
Confusion matrix: 
[[93  0  0  0  2]
 [ 1 22  0  0  0]
 [ 0  1 16  0  2]
 [ 0  2  5 19  0]
 [ 0  0  0  2 22]]
Score: 0.9197860962566845


In [56]:
none = np.array([0.26,0.07,0.1,0.32]).reshape(-1,4)
pred = RFC.predict(none)
print(pred)

index = np.array([0.9, 0.1, 0.4, 0.5]).reshape(-1,4)
pred = RFC.predict(index)
print(pred)

middle = np.array([1.6, 0.2, 1.5, 1.6]).reshape(-1,4)
pred = RFC.predict(middle)
print(pred)

fourth = np.array([1.9, 0.3, 0.6, 1.1]).reshape(-1,4)
pred = RFC.predict(fourth)
print(pred)

pinky = np.array([0.6, 0.2, 0.6, 1.1]).reshape(-1,4)
pred = RFC.predict(pinky)
print(pred)

[0]
[1]
[2]
[3]
[3]


## Multilayer Perceptron

In [51]:
MLP = MLPClassifier(hidden_layer_sizes=(200,), activation='relu', solver='adam', random_state=1, max_iter=1200)
MLP.fit(X_train, y_train)

probs = MLP.predict_proba(X_test)
preds = MLP.predict(X_test)
score = MLP.score(X_test, y_test)

print(f"Score: {score}")

Score: 0.9358288770053476


In [57]:
none = np.array([0.26,0.07,0.1,0.32]).reshape(-1,4)
pred = MLP.predict(none)
print(pred)

index = np.array([0.9, 0.1, 0.4, 0.5]).reshape(-1,4)
pred = MLP.predict(index)
print(pred)

middle = np.array([1.6, 0.2, 1.5, 1.6]).reshape(-1,4)
pred = MLP.predict(middle)
print(pred)

fourth = np.array([1.9, 0.3, 0.6, 1.1]).reshape(-1,4)
pred = MLP.predict(fourth)
print(pred)

pinky = np.array([0.6, 0.2, 0.6, 1.1]).reshape(-1,4)
pred = MLP.predict(pinky)
print(pred)

[0]
[1]
[2]
[3]
[3]


### True Test Values

In [70]:
none = np.array([0.18,0.07,0.27,0.26]).reshape(-1,4)
pred = KNN.predict(none)
print(pred)

index = np.array([1.07, 0.08, 0.49, 0.61]).reshape(-1,4)
pred = MLP.predict(index)
print(pred)

middle = np.array([2.02, 0.12, 0.76, 1.42]).reshape(-1,4)
pred = MLP.predict(middle)
print(pred)

fourth = np.array([1.06, 0.16, 0.61, 1.25]).reshape(-1,4)
pred = MLP.predict(fourth)
print(pred)

pinky = np.array([0.55, 0.1, 0.28, 0.88]).reshape(-1,4)
pred = MLP.predict(pinky)
print(pred)

[0]
[1]
[2]
[3]
[4]
